# Antibody – SARS-CoV-2 Molecular Dynamics Setup and energy estimation tutorial

This tutorial aims to illustrate the process of setting up a simulation system containing an antibody and a SARS-CoV-2 protein, <br>
step by step, using the SWISS-MODEL, VMD and NAMD. The energy estimation is implemented using AutoDock Tools, the script with commands is provided.

The particular example used is **SARS-CoV-2 protein** (PDB code 7c01) and **antibody derived from COVID-19 patient.**

The complete antibody sequence:

Heavy chain<br>
EVQLVESGGGLIQPGGSLRLSCAASGFTVSSNYMSWVRQAPGKGLEWVSVIYSGGSTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYC<br>
ARDRVVYGMDVWGQGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPS<br>
SSLGTQTYICNVNHKPSNTKVDKKVEPKSC

Light chain<br>
DIQMTQSPSSLSASVGDRVTITCQASQDISNYLNWYQQKPGKAPKLLIYDASNLETGVPSRFSGSGSGTDFTFTISSLQPEDIATYYCHQYDNLPP<br>
TFITFGQGTRLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKH<br>
KVYACEVTHQGLSSPVTKSFNRGEC
# Settings

## Software used

[SWISS-MODEL:](https://swissmodel.expasy.org/) a fully automated protein structure homology-modelling server

[VMD:](https://www.ks.uiuc.edu/Research/vmd/) a molecular visualization program for displaying, animating, and analyzing large biomolecular systems using 3-D graphics<br>
and built-in scripting

[NAMD:](https://www.ks.uiuc.edu/Research/namd/) a parallel molecular dynamics code designed for high-performance simulation of large biomolecular systems

[Chimera:](https://www.cgl.ucsf.edu/chimera/) a program for the interactive visualization and analysis of molecular structures and related data

[AutoDock Tools:](http://autodock.scripps.edu/) is a suite of automated docking tools. Allows estimation of binding energy.
# Pipeline steps

1. Building an antibody from a sequence

2. Superposition of the structure of built antibody and SARS-CoV-2 protein with antibody-SARS-CoV structure (PDB code 2dd8)

3. Minimization of the energy of antibody-SARS-CoV-2 structure

4. Creating a solvent box, solvation and ionization in VMD

5. Minimization of system and heating 

6. Equilibration and MD production run

7. Energy estimation of antibody-SARS-CoV-2 interactions for the last 1 ns of MD 

1. Building an antibody from a sequence

Let's build antibody. We will do it separetely for heavy and light chain. Paste a sequence for heavy chain into "Target sequence" field on [SWISS-MODEL](https://swissmodel.expasy.org/interactive) webpage and click "BUILD MODEL". Repeat the same for the light chain. When the modelling will be completed you can download built models for heavy and light chains. Save them as model_heavy.pdb and model_light.pdb.
<h2><center> SWISS-MODEL interactive workspace</center></h2>
    
<img src="Swiss-Model.png" width="900" height="450">


2. Superposition of the structure of built antibody and SARS-CoV-2 protein with antibody-SARS-CoV-2 structure (PDB code 7c01)

Please use script SARS_AB_rename.py to rename chains in the built antibody.
```python
> python SARS_AB_rename.py
```
Now we need to build an antibody-SARS-CoV-2 structure. Let's model antibody-SARS-CoV-2 interactions similar to the interaction mode of SARS-CoV-2 with neutralizing antibody CB6 (PDB code 7c01).

Let's do superposition of already build antibody with CB6 antibody-RBD domain of SARS-CoV-2 complex in Chimera.<br>
Open Chimera GUI and load 7C01_Ab_RBD.pdb, model_heavy_renamed.pdb and model_light_renamed.pdb.<br>
Go to Tools -> Structure Comparison -> Match Maker and do superposition of chain C (heavy chain) of 7C01_Ab_RBD.pdb with chain H of antibody.
<img src="Chimera_Superposition.png" width="800" height="450">

Repeat the same for a light chain of antibody. Do superposition of chain D (light chain) of 7C01_Ab_RBD.pdb with chain L of antibody. Remove chains C, D and save the current structure as Complex.pdb, which will be our working structure of antibody-SARS-CoV-2 complex.

3. Minimization of the energy of antibody-SARS-CoV-2 structure

Let's generate psf file for our built antibody-SARS-CoV-2 complex using VMD and minimize the energy of the structure with NAMD.<br>
```bash
> cd your_working_dir
> path_to_VMD/vmd -dispdev text -eofexit < Createpsf.tcl > output.log 
For example:
> ~/../../Applications/VMD/VMD_1_9_3.app/Contents/MacOS/startup.command -dispdev text -eofexit < Createpsf.tcl > output.log 
> mkdir min
> path_to_NAMD/namd2 +p8 minimization_1.txt > min1.log
```
NAMD will output 'Antibody_min.coor' file with energetically minimized structure in 'min' folder.

4. Creating a solvent box, solvation and ionization in VMD

We will solvate a structure in a rectangular box such that the minimum distance to the edge of the box will be 10 Å under periodic boundary conditions in VMD. Na and Cl ions will be added to neutralize the protein charge, then further ions will be added to mimic a salt solution concentration of 0.15 M.<br> 

```bash
> ~/../../Applications/VMD/VMD_1_9_3.app/Contents/MacOS/startup.command -dispdev text -eofexit < Solvate_ionize.tcl > out_solv.log 
```

5. Minimization of the system and heating.<br>
Let's minimize our system. Type in the Terminal:
```bash
> path_to_NAMD/namd2 +p8 minimization_2.txt > min_2.log
> ~/../../Applications/VMD/VMD_1_9_3.app/Contents/MacOS/startup.command -dispdev text -eofexit < Box_size.tcl > out.log
```
Also we need to determine the water box size and coordinates of the center of the water box.
The box_size.sh script will output three coordinates, which are the size of the water box. <br>

```bash
> box_size.sh  # to add coordinates of the center of the water box to configuration files.
> 81.165
> 84.174
> 128.159
```
Please make sure that all three output coordinates above are similar to coordinates in the fileds 'Periodic Boundary Conditions', 'cellBasisVector1', 'cellBasisVector2' and 'cellBasisVector3' in heating.conf, equilibration.conf, md.conf and minimization_2.txt. If these coordinates are different, the mentioned fields should be modified accordingly.
```bash
> mkdir heating
> path_to_NAMD/namd2 +p8 heating.conf > heating.log
```

6. Equilibration and MD production run

Let's equilibrate our systems for 500000 steps (1ns) and run 10 ns of MD. To run jobs on a cluster use a script run_on_cluster.slurm.
```bash
> mkdir equil
> path_to_NAMD/namd2 +p8 heating.conf > heating.log
> mkdir md
> path_to_NAMD/namd2 +p8 md.conf > md.log
To run jobs on cluster:
> sbatch run_on_cluster.slurm
```

7. Energy estimation of antibody-SARS-CoV-2 interactions for the last 1 ns of MD 

We ran 10ns of MD, so we will have 100 frames in total. As we saved trajectories every 50000 steps, for the last 1ns of MD we will have 10 frames. Let's save coordinates of antibody-SARS-CoV-2 structure for the last 10 frames of MD.
Let's load our psf and trajectory files into VMD and save 10 structures that corresponds to the last 10 frames.<br>
In VMD TK Console:
```bash
> ~/../../Applications/VMD/VMD_1_9_3.app/Contents/MacOS/startup.command -dispdev text -eofexit < Process_traj.tcl > out_tr.log 
> ./Script.sh
> ~/../../Applications/VMD/VMD_1_9_3.app/Contents/MacOS/startup.command -dispdev text -eofexit < Createpsf_10Ab.tcl > output.log 
```

We saved 10 structures, frame_90.pdb-frame_99.pdb into folder 'frames' and created psf files for them.
Now we need to analyze each complex and calculate antibody-SARS-CoV-2 interaction energy for each structure.<br>
First, let's do energy minimization of each complex.<br>
Please change a path to NAMD in the script Min_all.sh.
```bash
./Min_all.sh
```
After minimization we need to convert output files into pdb format.<br>

```bash
> ~/../../Applications/VMD/VMD_1_9_3.app/Contents/MacOS/startup.command -dispdev text -eofexit < Save_pdb.tcl > output.log
}
```

Now we can estimate energy for our 10 structures. The energy estimation requires installation of MGLTools. <br>
Please modify path to your workig folder and path to MGLTools (http://mgltools.scripps.edu/downloads/mgltools-1-5-7rc1) in the script Energy.sh.
```bash
> ./Energy.sh
```
It will output file energy.txt, which contains frame id and energy of antibody-SARS-CoV-2 interactions for heavy and light chains.